## YOLO

### PyTorch 기반 물체인식 모델
- CNN, rCNN(Regions with CNN)
- https://github.com/ultralytics/ultralytics 참조
 
#### YOLOv5 이상 설치
```shell
    > pip install ultralytics
```

In [ ]:
# YOLO 설치
!pip install ultralytics

#### 콘솔에서 예측

In [ ]:
# 콘솔에서 예측
## yolo11n.pt - pretrained YOLO model
## 자동으로 yolo11n.pt 다운로드
## 웹 URL에 있는 이미지도 예측이 가능
!yolo predict model=yolo11n.pt source='https://ultralytics.com/images/bus.jpg'

##### 파이썬으로 예측

In [5]:
# YOLO 모듈 로드
from ultralytics import YOLO

In [6]:
# YOLO 클래스가 들어오는 모델의 버전에 따라 알아서 YOLO 예측모델 객체 생성
model = YOLO('./yolo11n.pt')

##### coco8.yaml
- https://github.com/ultralytics/assets/releases/download/v0.0.0/coco8.zip
- 위 내용대로 훈련을 시킨 결과 -> yolo11n.pt

In [ ]:
# coco8.yaml - YOLO 훈련에 사용할 데이터 정의파일
train_resluts = model.train(
    data='./coco8.yaml',
    epochs=100,
    imgsz=640,
    device='cuda:0'
)

#### 이미지 예측

In [11]:
result = model('./0000001.jpg')


image 1/1 c:\Source\iot-dataanalysis-2025\day08\0000001.jpg: 480x640 1 cat, 110.8ms
Speed: 4.0ms preprocess, 110.8ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)


In [1]:
# matplotlib 모듈 로드
import matplotlib.pyplot as plt
from PIL import Image

In [20]:
# 예측결과 이미지 저장
img = result[0].plot()
img_pil = Image.fromarray(img[..., ::-1])
img_pil.save('./predict_result.jpg')

### OpenCV
- Opensource Computer Vision 약자. 실시간 컴퓨터 비전(시각처리)을 목적으로 프로그래밍 라이브러리
- 인텔에서 2000년에 C, C++ 사용하기 위해서 개발
- 파이썬에서 사용할 수 있게 래핑

    ```shell
    > pip install opencs-python
    ```

In [21]:
# OpenCV 설치
!pip install opencv-python


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import cv2
cv2.__version__

'4.11.0'

In [24]:
img2 = cv2.imread('./predict_result.jpg')
img2.shape  # (464, 640, 3) -> (height, width, channel)

(464, 640, 3)

In [ ]:
# 윈도우 창 오픈
cv2.imshow('결과', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### YOLO 예측

In [32]:
img = cv2.imread('./0000002.jpg')
resized_img = cv2.resize(img, (640, 400))

result = model(resized_img                                                                                                                                                                  )
plots = result[0].plot()

cv2.imshow('predic_result', plots)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 416x640 1 cup, 1 bowl, 1 mouse, 1 book, 28.3ms
Speed: 6.3ms preprocess, 28.3ms inference, 5.3ms postprocess per image at shape (1, 3, 416, 640)


#### 동영상 플레이
- 라즈베리파이에서 동일하게 사용가능
- 라즈베리파이 웹캠 사용추천

In [46]:
# 비디오 파일 경로
video_path = './sample01.mp4'
output_path = './sample01_output.mp4'
count_path = './sample01_count.mp4'

In [35]:
# 동영상 플레이
cap = cv2.VideoCapture(video_path)      # 0 -> 웹캠이나 카메라 설치된 번호

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    cv2.imshow('Video play', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):   # q버튼을 누르면
        break

cap.release()   # 비디오를 해제
cv2.destroyAllWindows()

#### YOLO 실시간 예측

In [40]:
# 시간 모듈 로드
import time

In [ ]:
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)     # 동영상 FPS(Frame Per Second)
frame_time = 1.0 / fps              # 초단위로 변환
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))      # 1280
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))    # 720

# VideoWriter 객체 생성(동영상 화면에 그림, 글자를 그리기 위한 객체)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    start_time = time.time()    # 시작시간
    ret, frame = cap.read()
    if not ret: break

    # 객체 탐지
    results = model(frame)
    # 탐지 결과 그리기
    for result in results:
        detect_frame = result.plot()
    # 결과 프레임을 파일로 저장
    out.write(detect_frame)
    # 결과 표시
    cv2.imshow('YOLO Object Detection', detect_frame)
    cv2.imshow('Video play', frame)

    # 프레임간 실제 지연시간 계산
    elapsed_time = time.time() - start_time
    delay = max(int((frame_time - elapsed_time) * 1000), 1)

    if cv2.waitKey(1) & 0xFF == ord('q'):   # q버튼을 누르면
        break

cap.release()   # 비디오를 해제
out.release()
cv2.destroyAllWindows()


In [ ]:
# shapely 설치
!pip install shapely==2.0.1

In [ ]:
# lap 설치
!pip install lap

#### Car Counting
- 지정된 라인 아래로 내려오는 자동차 개수 카운팅

In [49]:
import cv2
from ultralytics.solutions import ObjectCounter

In [ ]:
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), 'Error reading video file'   # 파일이 열리지 않으면 경고

region_pounts = [(20, 400), (180, 400)]             # 라인수
fps = cap.get(cv2.CAP_PROP_FPS)                     # 동영상 FPS(Frame Per Second)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))      # 1280
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))    # 720

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(count_path, fourcc, fps, (width, height))

# 물체 인식 핵심 객체
counter = ObjectCounter(
    show = True,                    # 처리 중 화면에 디스플레이
    region = region_pounts,         # 라인위치
    model = './yolo11n.pt',         # YOLO11 모델
    # classes=[0, 2], 
    # tracker='botsort.yaml',
)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    results = counter(frame)
    out.write(results.plot_im)  # 여기 차이

cap.release()   # 비디오를 해제
out.release()
cv2.destroyAllWindows()